In [24]:
!pip install openpyxl

import openpyxl
import pandas as pd
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.2 MB/s eta 0:00:00


On va procéder ici au merging des bases de données. 

## Préparation des sites 

### Préparation de la base manuelle

In [25]:
# Obtenez le chemin du répertoire parent du répertoire src
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construisez le chemin complet vers votre fichier Excel
fichier_excel = os.path.join(base_dir, 'data', 'text', 'manuel.xlsx')

# Chargez le fichier Excel dans un DataFrame
sites = pd.read_excel(fichier_excel, sheet_name=0)

In [22]:
base_dir

'/home/onyxia/work/Conservative_mobilisation'